<a href="https://colab.research.google.com/github/amrios80/Endeporte/blob/main/endeporte_proyecto_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>